# Init

In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/wliao0504/code/clif/CLIF-MIMIC'

In [2]:
import numpy as np
import pandas as pd
import logging
import duckdb
import pandera as pa
from importlib import reload
import src.utils as utils
from typing import Dict, List
# reload(utils)
from src.utils import construct_mapper_dict, fetch_mimic_events, load_mapping_csv, \
    get_relevant_item_ids, find_duplicates, rename_and_reorder_cols, save_to_rclif, \
    convert_and_sort_datetime, setup_logging, con, REPO_ROOT, mimic_table_pathfinder, \
    search_mimic_items, read_from_rclif
from src.utils_qa import all_null_check

loaded configuration from /Users/wliao0504/code/clif/CLIF-MIMIC/src/../config/config.json


# Dev

In [3]:
import src.tables.ecmo_mcs as ecmo_mcs
reload(ecmo_mcs)

2025-05-13 12:09:52,418 - INFO - initialized logging at logs/etl.log
2025-05-13 12:09:52,420 - INFO - initialized logging at logs/etl.log


<module 'src.tables.ecmo_mcs' from '/Users/wliao0504/code/clif/CLIF-MIMIC/src/tables/ecmo_mcs.py'>

In [4]:
ecmo_mcs._main()

2025-05-13 12:09:52,424 - INFO - starting to build clif ecmo_mcs table -- 
2025-05-13 12:09:52,443 - INFO - initialized logging at logs/etl.log
2025-05-13 12:09:52,444 - INFO - Note: Hamilton collects completely anonymous data about usage. This will help us improve Hamilton over time. See https://hamilton.dagworks.io/en/latest/get-started/license/#usage-analytics-data-privacy for details.
2025-05-13 12:09:52,451 - INFO - parsing the mapping files to identify relevant items and fetch corresponding events...
2025-05-13 12:09:52,452 - INFO - querying the d_items table to identify which event tables to be separately queried for 228 items
2025-05-13 12:09:52,456 - INFO - identified 3 event tables to be separately queried: ['chartevents', 'datetimeevents', 'procedureevents']
2025-05-13 12:09:52,456 - INFO - fetching events from chartevents table for 203 items
2025-05-13 12:09:55,193 - INFO - fetched 1159127 events from chartevents table for 203 items
2025-05-13 12:09:55,194 - INFO - fetching

In [5]:
test_out = ecmo_mcs._test()

2025-05-13 12:09:56,234 - INFO - testing all...
2025-05-13 12:09:56,235 - INFO - initialized logging at logs/etl.log
2025-05-13 12:09:56,249 - INFO - parsing the mapping files to identify relevant items and fetch corresponding events...
2025-05-13 12:09:56,250 - INFO - querying the d_items table to identify which event tables to be separately queried for 228 items
2025-05-13 12:09:56,255 - INFO - identified 3 event tables to be separately queried: ['chartevents', 'datetimeevents', 'procedureevents']
2025-05-13 12:09:56,255 - INFO - fetching events from chartevents table for 203 items
2025-05-13 12:09:58,554 - INFO - fetched 1159127 events from chartevents table for 203 items
2025-05-13 12:09:58,554 - INFO - fetching events from datetimeevents table for 16 items
2025-05-13 12:09:58,554 - WARNING - datetimeevents not yet supported, thus returning all columns
2025-05-13 12:09:58,652 - INFO - fetched 41872 events from datetimeevents table for 16 items
2025-05-13 12:09:58,653 - INFO - fetch

{'schema_tested': True}


In [6]:
clif_ecmo = read_from_rclif("ecmo_mcs")

In [7]:
clif_ecmo

,hospitalization_id,recorded_dttm,device_name,device_category,mcs_group,side,device_metric_name,device_rate,flow,sweep,fdo2
0,20044789,2180-08-20 01:00:00+00:00,VV,Other,ECMO,right,RPM,3400.0,3.20,3.0,100.0
1,20044789,2180-08-20 02:00:00+00:00,VV,Other,ECMO,right,RPM,3900.0,3.71,3.0,100.0
2,20044789,2180-08-20 02:30:00+00:00,VV,Other,ECMO,right,RPM,4000.0,3.84,NaN,NaN
3,20044789,2180-08-20 03:00:00+00:00,VV,Other,ECMO,right,RPM,4000.0,3.80,3.0,100.0
4,20044789,2180-08-20 04:00:00+00:00,VV,Other,ECMO,right,RPM,4000.0,3.77,3.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
93394,29984509,2158-11-01 13:00:00+00:00,<NA>,Other,None,None,RPM,5600.0,5.20,NaN,NaN
93395,29984509,2158-11-01 15:00:00+00:00,<NA>,Other,None,None,RPM,5600.0,5.30,NaN,NaN
93396,29984509,2158-11-01 18:00:00+00:00,<NA>,Other,None,None,RPM,5600.0,5.20,NaN,NaN
93397,29984509,2158-11-01 22:00:00+00:00,<NA>,Other,None,None,RPM,5600.0,5.20,NaN,NaN


# Validate